In [ ]:
import pickle
import numpy as np
from dnn_model import DNNModel
import torch
import tqdm

In [56]:
from importlib import reload
import dnn_model
reload(dnn_model)

<module 'dnn_model' from '/media/hp/New Volume1/Programming/LearnROS/alpha-RACER/car_ros2/car_ros2/notebooks/dnn_model.py'>

In [57]:
DATA_FILE = '../data/data_large_multi_mpc(1).pkl'
MODEL_DIMENSIONS = [39, 128, 128, 64, 1]
OPPONENTS = {
    0: [2, 1],
    1: [2, 0],
    2: [0, 1]
}
HIDDEN_WINDOW = 100
DISCOUNT_FACTOR = 0.99
N = 3
LR = 0.001
EPOCHS = 500
BATCH_SIZE = 32

data = pickle.load(open(DATA_FILE, 'rb'))

In [58]:
# data = np.array(data[:-1])  # Incomplete last episode
data.shape

(45, 500, 39)

In [59]:
def fix_difference(t):
    t = (t > 75.0435) * (t - 150.087) + (t < -75.0435) * (t + 150.087) + (t <= 75.0435) * (t >= -75.0435) * t
    return t

In [65]:
X = data[:,:,:].copy()
X[:,:,0] = data[:,:,2] - data[:,:,1]
X[:,:,1] -= data[:,:,0]
X[:,:,2] -= data[:,:,0]
for i in range(N):
    X[:,:,i] = fix_difference(X[:,:,i])

X = torch.tensor(X, dtype=torch.float32)

Y = torch.zeros((N, X.shape[0], X.shape[1]-1))
for i in range(N):
    Y[i, :, :] = torch.maximum(X[:, 1:, OPPONENTS[i][0]], X[:, 1:, OPPONENTS[i][1]]) - torch.maximum(X[:, :-1, OPPONENTS[i][0]], X[:, :-1, OPPONENTS[i][1]])
    Y[i, :, :] = fix_difference(Y[i, :, :])

rewards = torch.zeros(N, X.shape[0], Y.shape[2]-HIDDEN_WINDOW)
masks = []

for j in range(N):
    for i in range(0, HIDDEN_WINDOW):
        rewards[j, :, :] += (DISCOUNT_FACTOR**i) * Y[j, :, i:i+Y.shape[2]-HIDDEN_WINDOW] 

    masks.append(torch.tensor((rewards[j] < 600.) * (rewards[j] >= -100.)))

/tmp/ipykernel_305307/463542353.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masks.append(torch.tensor((rewards[j] < 600.) * (rewards[j] >= -100.)))


In [63]:
models = [DNNModel(MODEL_DIMENSIONS) for _ in range(N)]
optimizers = [torch.optim.Adam(model.parameters(), lr=LR) for model in models]
loss_fn = torch.nn.MSELoss()

In [ ]:
for e in tqdm.tqdm(range(EPOCHS)):
    total_loss = [0. for _ in range(N)]
    batch_tqdm = tqdm.tqdm(range(0, len(X)-BATCH_SIZE+1, BATCH_SIZE), leave=False)
    for j in batch_tqdm:
        x_batch = X[j:j+BATCH_SIZE, :-HIDDEN_WINDOW-1, :]
        for i in range(N):
            y_batch = rewards[i, j:j+BATCH_SIZE, :]
            mask = masks[i][j:j+BATCH_SIZE, :]

            optimizers[i].zero_grad()
            preds = models[i](x_batch).squeeze()
            loss = loss_fn(preds * mask, y_batch * mask)
            loss.backward()
            optimizers[i].step()

            total_loss[i] += loss.item()

torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399, 39]) torch.Size([32, 399]) torch.Size([32, 399])
torch.Size([32, 399,

KeyboardInterrupt: 